In [6]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
import requests
from config import weather_api_key
import time
from datetime import datetime

In [10]:
# Create a set of random latitude and longitude combinations.

lats = np.random.uniform(low=-90.00,high=90.00,size=1500)
lngs = np.random.uniform(low=-180.00,high=180.00,size=1500)

lat_lngs = zip(lats,lngs)

lat_lngs

In [11]:
coordinates =list(lat_lngs)

In [12]:
cities =[]

for coordinate in coordinates:
    
    city = citipy.nearest_city(coordinate[0],coordinate[1]).city_name

    if city not in cities:

        cities.append(city)

len(cities)

618

In [13]:
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key


In [15]:
city_data = []

print('Beginning Data Retrieval     ')
print('-----------------------------')

# Create counters

record_count = 1
set_count = 1


# Loop through all the cities in the list

for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes

    if (i % 50 == 0 and i >= 50):

        set_count += 1
        record_count = 1
        time.sleep(60)

    # Create endpoint URL with each city

    city_url = url + "&q=" + city.replace(" ","+")


    # Log the url, record, and set numbers and the city

    print(f'Processing Record {record_count} of Set {set_count} | {city}')

    record_count += 1

    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | belaya gora
Processing Record 2 of Set 1 | ribeira grande
Processing Record 3 of Set 1 | kapaa
Processing Record 4 of Set 1 | hilo
Processing Record 5 of Set 1 | hambantota
Processing Record 6 of Set 1 | albany
Processing Record 7 of Set 1 | tidore
City not found. Skipping...
Processing Record 8 of Set 1 | kruisfontein
Processing Record 9 of Set 1 | voykove
Processing Record 10 of Set 1 | nanortalik
Processing Record 11 of Set 1 | ushuaia
Processing Record 12 of Set 1 | rikitea
Processing Record 13 of Set 1 | atuona
Processing Record 14 of Set 1 | lazaro cardenas
Processing Record 15 of Set 1 | punta arenas
Processing Record 16 of Set 1 | blagoyevo
Processing Record 17 of Set 1 | alexandria
Processing Record 18 of Set 1 | norman wells
Processing Record 19 of Set 1 | poum
Processing Record 20 of Set 1 | sarab
Processing Record 21 of Set 1 | benin
Processing Record 22 of Set 1 | carnarvon
Processin

In [17]:
city_data_df = pd.DataFrame(city_data)

city_data_df.head(5)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Belaya Gora,68.5333,146.4167,54.97,56,98,11.54,RU,2022-07-16 22:08:14
1,Ribeira Grande,38.5167,-28.7000,71.80,86,1,5.86,PT,2022-07-16 22:08:14
2,Kapaa,22.0752,-159.3190,84.18,78,100,18.41,US,2022-07-16 22:04:55
3,Hilo,19.7297,-155.0900,83.28,91,100,16.11,US,2022-07-16 22:03:32
4,Hambantota,6.1241,81.1185,79.14,100,20,8.05,LK,2022-07-16 22:08:14


In [19]:
new_column_order = ['City','Cloudiness','Country','Date','Humidity','Lat','Lng','Max Temp','Wind Speed']

city_data_df = city_data_df[new_column_order]

city_data_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Belaya Gora,98,RU,2022-07-16 22:08:14,56,68.5333,146.4167,54.97,11.54
1,Ribeira Grande,1,PT,2022-07-16 22:08:14,86,38.5167,-28.7000,71.80,5.86
2,Kapaa,100,US,2022-07-16 22:04:55,78,22.0752,-159.3190,84.18,18.41
3,Hilo,100,US,2022-07-16 22:03:32,91,19.7297,-155.0900,83.28,16.11
4,Hambantota,20,LK,2022-07-16 22:08:14,100,6.1241,81.1185,79.14,8.05


In [21]:
# Output file
output_data_file = 'weather_data/cities.csv'

# export the df to csv
city_data_df.to_csv(output_data_file,index_label="City_ID")